In [ ]:
#https://github.com/facebookresearch/detectron2.git вставляю этот репозиторий
# pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [1]:
import pandas as pd
import os
import detectron2
import shutil
import cv2
import requests
import random
from io import BytesIO
from PIL import Image
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import numpy as np

In [2]:
data = pd.read_csv('preprocessed_data.csv')

In [3]:
data['id'] = range(0, len(data))

In [149]:
sorted_dir = 'sorted'
if not os.path.exists(sorted_dir):
    os.makedirs(sorted_dir)

In [9]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

interior_objects = ['chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop',  'remote', 
'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'teddy bear', 'hair drier', 'toothbrush'
] 

outdoor_objects = ['bird','person', 'bicycle', 'car', 'motorcycle', 'airplane', 
'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog','umbrella']

def check_objects(instances, class_names):
    interior_found = False
    outdoor_found = False

    for i in range(len(instances.pred_classes)):
        class_id = int(instances.pred_classes[i])
        class_name = class_names[class_id]

        if class_name in interior_objects:
            interior_found = True
        if class_name in outdoor_objects:
            outdoor_found = True
    return interior_found, outdoor_found

for idx, row in data.iterrows():
    folder_name = str(row['id'])
    new_folder_path = os.path.join(sorted_dir, folder_name)
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)

    photo_urls = row['photos'].split(',')
    valid_images = []

    for i, url in enumerate(photo_urls):
        try:
            response = requests.get(url.strip(), timeout=10)
            if response.status_code == 200:
                img = Image.open(BytesIO(response.content))
                img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
                outputs = predictor(img)
                instances = outputs["instances"]
                class_names = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes
                interior_found, outdoor_found = check_objects(instances, class_names)

                if interior_found and not outdoor_found:
                    valid_images.append((url, img))

        except Exception as e:
            print(f"Ошибка при скачивании {url}: {e}")
            continue  

    if len(valid_images) > 5:
        valid_images = random.sample(valid_images, 5)

    for i, (url, img) in enumerate(valid_images):
        img_name = f"{folder_name}_photo_{i+1}.jpg"
        img_path = os.path.join(new_folder_path, img_name)
        cv2.imwrite(img_path, img)  

print("Фотографии успешно отсортированы.")

Фотографии успешно отсортированы.


In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("LVISv0.5-InstanceSegmentation/mask_rcnn_R_101_FPN_1x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("LVISv0.5-InstanceSegmentation/mask_rcnn_R_101_FPN_1x.yaml")

predictor = DefaultPredictor(cfg)
sorted_dir = 'sortlv'
if not os.path.exists(sorted_dir):
    os.makedirs(sorted_dir)
    
interior_objects = [
'air_conditioner',  'alarm_clock',  'armchair', 'armoire', 'backpack', 'handbag', 'suitcase',  'basket', 'bath_towel',
'bathrobe', 'bathtub',  'battery', 'bed',  'bedspread',  'blanket', 'blazer', 'blender', 'book', 'bookcase',  'bottle',   
'bunk_bed',  'cabin_car', 'cabinet', 'locker', 'candle', 'candle_holder', 'chair', 'chandelier',  'clothes_hamper',  'coat_hanger', 
'coffee_table', 'sofa_bed', 'cooker', 'cooking_utensil', 'cornice',  'crock_pot',  'cup', 'cupboard', 'curtain', 'cushion', 'desk',  
'dining_table',  'dishrag', 'dishwasher', 'dishwasher_detergent', 'doorknob', 'doormat',  'dresser','refrigerator', 'fan', 'faucet', 
'fireplace',  'folding_chair', 'food_processor',    'hair_dryer',  'headboard',  'headset',  'heater', 'highchair', 'hotplate', 'iron_(for_clothing)',
'ironing_board', 'jacket', 'kitchen_sink', 'kitchen_table', 'lamp', 'lampshade', 'lightbulb', 'loveseat',   'mattress',  'microwave_oven',  'mirror','ottoman', 
'oven', 'shower_head', 'shower_cap', 'shower_curtain',  'sink', 'sofa',  'table_lamp', 'teacup', 'teakettle', 'teapot', 'toaster', 'toaster_oven', 'toilet', 
'toilet_tissue',  'towel', 'towel_rack', 'wall_clock', 'wall_socket', 'wardrobe', 'washbasin', 'automatic_washer','water_bottle', 'water_cooler', 'water_faucet', 
'water_heater', 'water_jug',  'window_box_(for_plants)'
]

outdoor_objects = [
'person','car_(automobile)','elevator_car', 'car_battery','ladder', 'Christmas_tree', 'deck_chair','bench'
]

def check_objects(instances, class_names):
    interior_found = False
    outdoor_found = False

    for i in range(len(instances.pred_classes)):
        class_id = int(instances.pred_classes[i])
        class_name = class_names[class_id]

        if class_name in interior_objects:
            interior_found = True
        if class_name in outdoor_objects:
            outdoor_found = True
            
    return interior_found, outdoor_found

for idx, row in data.iterrows():
    folder_name = str(row['id'])
    new_folder_path = os.path.join(sorted_dir, folder_name)
    
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)

    photo_urls = row['photos'].split(',')
    valid_images = []

    for i, url in enumerate(photo_urls):
        try:
            response = requests.get(url.strip(), timeout=10)
            if response.status_code == 200:
                img = Image.open(BytesIO(response.content))
                img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
                outputs = predictor(img)
                instances = outputs["instances"]
                class_names = MetadataCatalog.get("lvis_v1_train").thing_classes
                interior_found, outdoor_found = check_objects(instances, class_names)

                if interior_found and not outdoor_found:
                    valid_images.append((url, img))

        except Exception as e:
            print(f"Ошибка при скачивании {url}: {e}")
            continue  

    if len(valid_images) > 5:
        valid_images = random.sample(valid_images, 5)

    for i, (url, img) in enumerate(valid_images):
        img_name = f"{folder_name}_photo_{i+1}.jpg"
        img_path = os.path.join(new_folder_path, img_name)
        cv2.imwrite(img_path, img)  

print("Фотографии успешно отсортированы.")

In [ ]:
def count_folders_with_exact_images(root_folder, image_count):
 
    exact_images_folders = []  
    for folder_name in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder_name)
        if os.path.isdir(folder_path):
            images = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
            if len(images) == image_count:
                exact_images_folders.append(folder_name)
        return sorted(exact_images_folders)

def copy_and_replace_folders(sorted_folder, sortlv_folder, new_sorted_folder, sorted_exact, sortlv_exact):

    if not os.path.exists(new_sorted_folder):
        os.makedirs(new_sorted_folder)
    
    for folder in sorted_exact:
        sorted_folder_path = os.path.join(sorted_folder, folder)
        new_sorted_folder_path = os.path.join(new_sorted_folder, folder)
        if os.path.isdir(sorted_folder_path):
            shutil.copytree(sorted_folder_path, new_sorted_folder_path)
    
    for folder in sortlv_exact:
        if folder not in sorted_exact:
            sortlv_folder_path = os.path.join(sortlv_folder, folder)
            new_sorted_folder_path = os.path.join(new_sorted_folder, folder)

            if os.path.isdir(sortlv_folder_path):
                shutil.copytree(sortlv_folder_path, new_sorted_folder_path)
                print(f"Папка '{folder}' заменена в 'new_sorted'.")

sorted_folder = "sorted"
sortlv_folder = "sortlv"
new_sorted_folder = "new_sorted"

image_count = 5

sorted_exact = count_folders_with_exact_images(sorted_folder, image_count)
sortlv_exact = count_folders_with_exact_images(sortlv_folder, image_count)

copy_and_replace_folders(sorted_folder, sortlv_folder, new_sorted_folder, sorted_exact, sortlv_exact)

In [ ]:
data['id'] = data.index

In [ ]:
folder_path = "new_sorted"

folder_names = [int(folder) for folder in os.listdir(new_sorted_folder) 
                if os.path.isdir(os.path.join(new_sorted_folder, folder)) and folder.isdigit()]

folder_names.sort()
data_filtered = data[data['id'].isin(folder_names)]
data_filtered.to_csv('preprocessed_data.csv')